In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score as sil_, calinski_harabasz_score as calinski_
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
import re
from datetime import datetime, timezone
from sklearn.preprocessing import OneHotEncoder
import ipaddress
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from collections import Counter
import plotly.express as px
import matplotlib.dates as mdates
import folium
import pytz
from timezonefinder import TimezoneFinder
import pickle


C:\Users\luca.lucchina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/omnipot.csv')

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df.isna().sum()

src_port                0
dst_port                0
timestamp               0
src_as               2657
src_city                5
src_country             0
src_countryCode        42
src_geo_str             0
src_ip                  0
src_proxy               0
src_regionName        171
dst_as                  0
dst_city                0
dst_country             0
dst_countryCode         0
dst_geo_str             0
dst_ip                  0
dst_proxy               0
dst_regionName          0
protocol                0
payload_4kb_hex    209362
dtype: int64

In [5]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

In [6]:
# filling payload nan with "" the ndrop nan that are not in the payload column as empty payloads are valid
df.fillna({"payload_4kb_hex":""}, inplace=True)
df.dropna(inplace=True)

In [7]:
def get_ip_encoding(ipv4):
    oct1, oct2, oct3, oct4 = ipv4.split('.')
    return  int(oct1), int(oct2), int(oct3), int(oct4)

df[['src_oct1', 'src_oct2', 'src_oct3', 'src_oct4']] = df["src_ip"].apply(get_ip_encoding).apply(pd.Series)
df[['dst_oct1', 'dst_oct2', 'dst_oct3', 'dst_oct4']] = df["dst_ip"].apply(get_ip_encoding).apply(pd.Series)

In [8]:
df["src_geo_str"] = df["src_geo_str"].str.split("|").apply(lambda lst: [float(x) for x in lst])
df["dst_geo_str"] = df["dst_geo_str"].str.split("|").apply(lambda lst: [float(x) for x in lst])

In [9]:
from scipy.stats import entropy

def get_payload_features(payload):
    default = {'length': 0, 'unique_bytes': 0, 'entropy': 0.0,
               'mean_byte': 0.0, 'std_byte': 0.0}
    if not isinstance(payload, str) or len(payload) == 0:
        return default
    try:
        data = bytes.fromhex(payload)
    except ValueError:
        return default
    length = len(data)
    if length == 0:
        return default
    arr = np.frombuffer(data, dtype=np.uint8)
    counts = np.bincount(arr, minlength=256)
    probs = counts / length

    probs = probs[probs > 0]
    ent = entropy(probs, base=2)
    mean = float(arr.mean())
    std = float(arr.std())
    return {
        'length': length,
        'unique_bytes': int(np.count_nonzero(counts)),
        'entropy': float(ent),
        'mean_byte': mean,
        'std_byte': std
    }

payload_feats = ['pl_length', 'pl_unique_bytes', 'pl_entropy', 'pl_mean_byte', 'pl_std_byte']
df[payload_feats] = df["payload_4kb_hex"].apply(get_payload_features).apply(pd.Series)

In [ ]:
def decode_payload(hex_str: str, protocol: str) -> str:
    if not hex_str or not isinstance(hex_str, str):
        return ""
    
    try:
        raw_bytes = bytes.fromhex(hex_str)
        if not raw_bytes:
            return ""

        # SSH 
        if protocol == 'ssh':
            # Clean SSH banners
            if raw_bytes.startswith(b'SSH-'):
                banner = raw_bytes.decode('utf-8', errors='ignore').split('\n')[0]
                return banner.strip()
            
            # Detect HTTP requests on SSH port
            if raw_bytes.startswith((b'GET ', b'POST ', b'HEAD ', b'PUT ')):
                try:
                    return "HTTP_OVER_SSH: " + raw_bytes.decode('utf-8').split('\r\n')[0]
                except:
                    return "HTTP_OVER_SSH_BINARY"
            
            # Detect binary commands
            try:
                text = raw_bytes.decode('utf-8', errors='ignore')
                if any(cmd in text.lower() for cmd in ['wget', 'curl', 'chmod', 'sh -c']):
                    return "SSH_CMD: " + ' '.join(text.split())[:200]
            except:
                pass
            
            return f"SSH_BINARY_{len(raw_bytes)}B"

        # HTTP/HTTPS
        elif protocol in ['http', 'https']:
            # Reject SSH banners in HTTP
            if raw_bytes.startswith(b'SSH-'):
                return "SSH_OVER_HTTP"
            
            # Handle valid HTTP
            if raw_bytes.startswith((b'GET ', b'POST ', b'HEAD ', b'PUT ')):
                try:
                    first_line = raw_bytes.decode('utf-8').split('\r\n')[0]
                    return f"HTTP: {first_line}"
                except:
                    return "HTTP_BINARY"
            
            # Handle SSL/TLS
            if len(raw_bytes) > 0 and raw_bytes[0] == 0x16:
                return "SSL_HANDSHAKE"
            
            # Other cases
            try:
                text = raw_bytes.decode('utf-8', errors='ignore')
                if text.strip():
                    return f"HTTP_TEXT: {text[:200]}"
            except:
                pass
            
            return f"HTTP_BINARY_{len(raw_bytes)}B"

        # SMB 
        elif protocol == 'smb':
            if len(raw_bytes) > 8:
                # SMB command mapping
                smb_commands = {
                    0x00: "SMB_NEGOTIATE",
                    0x73: "SMB_NEGOTIATE",  # Common variant
                    0x75: "SMB_TREE_CONNECT",
                    0x1d: "SMB_SESSION_SETUP",
                    0x25: "SMB_CREATE",
                    0x2e: "SMB_IOCTL",
                    0x32: "SMB_READ",
                    0x0b: "SMB_TREE_DISCONNECT"
                }
                cmd = smb_commands.get(raw_bytes[4], f"SMB_UNKNOWN_{raw_bytes[4]}")
                return cmd if cmd != "SMB_NEGOTIATE" or raw_bytes[4:8] == b'\xffSMB' else "SMB_INVALID"
            return "SMB_EMPTY"

        # Other Protocols 
        else:
            try:
                text = raw_bytes.decode('utf-8', errors='ignore')
                if text.strip():
                    return f"{protocol}_TEXT: {text[:200]}"
            except:
                pass
            return f"{protocol}_BINARY_{len(raw_bytes)}B"

    except Exception as e:
        return f"DECODE_ERROR_{str(e)}"

# Reapply decoding
df['payload_decoded'] = df.apply(
    lambda row: decode_payload(row['payload_4kb_hex'], row['protocol']), 
    axis=1
)

: 

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

def get_code_embeddings(payloads, model_name='microsoft/codebert-base', batch_size=16):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    embeddings = []
    for i in range(0, len(payloads), batch_size):
        batch = payloads[i:i + batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_vectors = outputs.last_hidden_state[:, 0, :]
            embeddings.extend(cls_vectors.cpu().numpy())

    return embeddings

# Usage
payloads = df['payload_decoded'].astype(str).tolist()
vectors = get_code_embeddings(payloads)
cosine_sim_matrix = cosine_similarity(vectors)

C:\Users\luca.lucchina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\luca.lucchina\.cache\huggingface\hub\models--microsoft--codebert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Stor

In [ ]:
from hdbscan import HDBSCAN
clusterer = HDBSCAN(min_cluster_size=10000).fit(vectors)
df['payload_cluster'] = clusterer.labels_

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
reduced = PCA(n_components=2).fit_transform(vectors)
plt.scatter(reduced[:,0], reduced[:,1], c=df['payload_cluster'], cmap='tab10')


In [ ]:
# Keep only payloads that are long enough and have some entropy
df_anom = df[(df['pl_length'] > 10) & (df['pl_entropy'] > 1.0)]

from sklearn.ensemble import IsolationForest

clf = IsolationForest(contamination=0.01, random_state=42)
clf.fit([vectors[i] for i in df_anom.index])
df_anom['anomaly_score'] = clf.decision_function([vectors[i] for i in df_anom.index])
df_anom['is_anomaly'] = clf.predict([vectors[i] for i in df_anom.index]) == -1

df.loc[df_anom.index, 'anomaly_score'] = df_anom['anomaly_score']
df.loc[df_anom.index, 'is_anomaly'] = df_anom['is_anomaly']
df['is_anomaly'] = df['is_anomaly'].fillna(False)  # others default to not anomalous
